In [1]:
from torch.utils.data import DataLoader, dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import image_dataset

In [2]:
#set manual seed
torch.manual_seed(42)

# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset = image_dataset.ImageDataset()
print(len(dataset))
for i in range(len(dataset)):
    sample = dataset[i]
    print(f"Image: {sample[0]}, Label: {sample[1]}")

96
Image: <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512 at 0x130608B90>, Label: 0
Image: <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512 at 0x130AEC550>, Label: 0
Image: <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512 at 0x1107BD6D0>, Label: 0
Image: <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512 at 0x12FDED590>, Label: 0
Image: <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512 at 0x11079E910>, Label: 0
Image: <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512 at 0x130AEC550>, Label: 0
Image: <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512 at 0x12FDED590>, Label: 0
Image: <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512 at 0x11079E910>, Label: 0
Image: <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512 at 0x1305F1D50>, Label: 0
Image: <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512 at 0x12FDED590>, Label: 0
Image: <PIL.PngImagePlugin.PngImageFile image mode=L size=512x512 at 0x130608

In [4]:
#define the generator network
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()

    def forward(self, x):
        return self.tanh(self.fc2(self.relu(self.fc1(x))))
    
#define the discriminator network
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.fc2(self.relu(self.fc1(x))))